In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std

plt.rcParams.update({'font.size': 14})

## Laborator: exerciţii şi probleme

__Q1__. Pentru swiss dataset, realizați modelul de regresie liniară pentru Fertility în funcție de Agriculture și Education. Remarcați intercept și cei doi coeficienți.

Mai departe arătați cum coeficientul regresorului $x_1$ (Agriculture) este de fapt coeficientul regresiei caracteristice răspunsului $y$ (Fertility) și regresorului $x_2$ (Education) din care contribuția celuilalt regresor ($x_1$) a fost deja înlăturată.

__Hint1__: contribuția unui regresor nu mai este prezentă în reziduul asociat, și se înlătură folosind regresia liniară (revedeți slide-urile 42-end din cursul 7).

__Hint2__: folosiți proprietatea .resid a modelului deja potrivit.

In [2]:
swiss = pd.read_csv('swiss.csv')
swiss.columns = ['District', 'Fertility', 'Agriculture', 'Examination', 'Education', 'Catholic', 
                 'InfantMortality']
swiss.head()

,District,Fertility,Agriculture,Examination,Education,Catholic,InfantMortality
0,Courtelary,80.2,17.0,15,12,9.96,22.2
1,Delemont,83.1,45.1,6,9,84.84,22.2
2,Franches-Mnt,92.5,39.7,5,5,93.40,20.2
3,Moutier,85.8,36.5,12,7,33.77,20.3
4,Neuveville,76.9,43.5,17,15,5.16,20.6


__Q2__. Pentru setul mtcars, considerați variabila categorială 'număr de cilindri'.

a) Ridicați diagrama pair plot.

b) Calculați coeficienții de regresie. Există variabile care par să explice consumul?

c) Ridicați pe un scatter plot regresia mpg funcție de horsepower.

d) În funcție și de numărul de cilindri, realizați două linii de regresie dacă presupunem că nu există interacțiune între horsepower și numărul de cilindri.

e) În funcție și de numărul de cilindri, realizați două linii de regresie dacă presupunem acum că există totuși interacțiune.

In [3]:
mtcars = pd.read_csv('mtcars.csv')
mtcars.head()

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
